# Building and deploying machine learning solutions with Vertex AI: Challenge Lab

## Scenario

* Train a TensorFlow model locally in a hosted [**Vertex Notebook**](https://cloud.google.com/vertex-ai/docs/general/notebooks?hl=sv).
* Containerize your training code with [**Cloud Build**](https://cloud.google.com/build) and push it to [**Google Cloud Artifact Registry**](https://cloud.google.com/artifact-registry).
* Define a pipeline using the [**Kubeflow Pipelines (KFP) V2 SDK**](https://www.kubeflow.org/docs/components/pipelines/sdk/v2/v2-compatibility) to train and deploy your model on [**Vertex Pipelines**](https://cloud.google.com/vertex-ai/docs/pipelines).
* Query your model on a [**Vertex Endpoint**](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) using online predictions.

**NOTE: Make sure you have installed the required packages for the lab as specified in the Task 2 > step 3 of the lab instructions.**

### Define constants

In [ ]:
# Add installed library dependencies to Python PATH variable.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

In [ ]:
# Retrieve and set PROJECT_ID and REGION environment variables.
# TODO: Fill in the PROJECT_ID and REGION provided in the lab manual.
from dotenv import load_dotenv
import os
load_dotenv()

PROJECT_ID = os.environ.get('PROJECT_ID')
REGION = "us-central1"

In [ ]:
# TODO: Create a globally unique Google Cloud Storage bucket for artifact storage.
GCS_BUCKET = "gs://vetexai-bucket-test"

In [ ]:
!gsutil mb -l $REGION $GCS_BUCKET

### Import libraries

In [ ]:
import datetime

import tensorflow as tf

from official.nlp import optimization  

from google.cloud import aiplatform

# # Install pydot and graphviz
# !pip install pydot
# !sudo apt install graphviz -y

from trainer.model import load_datasets, build_text_classifier

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Import dataset

In [ ]:
DATA_URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
LOCAL_DATA_DIR = "."

In [ ]:
# DATASET_DIR = download_data(data_url=DATA_URL, local_data_dir=LOCAL_DATA_DIR)
DATASET_DIR = 'aclImdb'


In [ ]:
# Create a dictionary to iteratively add data pipeline and model training hyperparameters.
HPARAMS = {
    # Set a random sampling seed to prevent data leakage in data splits from files.
    "seed": 42,
    # Number of training and inference examples.
    "batch-size": 32
}

In [ ]:
raw_train_ds, raw_val_ds, raw_test_ds = load_datasets(DATASET_DIR, HPARAMS)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
CLASS_NAMES = raw_train_ds.class_names

train_ds = raw_train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = raw_test_ds.prefetch(buffer_size=AUTOTUNE)

Let's print a few example reviews:

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review {i}: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({CLASS_NAMES[label]})')

In [ ]:
HPARAMS.update({
    # TF Hub BERT modules.
    "tfhub-bert-preprocessor": "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    "tfhub-bert-encoder": "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2",
})

In [ ]:
HPARAMS.update({
    # Model training hyperparameters for fine tuning and regularization.
    "epochs": 0,
    "initial-learning-rate": 3e-5,
    "dropout": 0.1 
})

In [ ]:
epochs = HPARAMS['epochs']
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
n_train_steps = steps_per_epoch * epochs
n_warmup_steps = int(0.1 * n_train_steps)    

OPTIMIZER = optimization.create_optimizer(init_lr=HPARAMS['initial-learning-rate'],
                                          num_train_steps=n_train_steps,
                                          num_warmup_steps=n_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
model = build_text_classifier(HPARAMS, OPTIMIZER)

In [ ]:
# Visualize your fine-tuned BERT sentiment classifier.
tf.keras.utils.plot_model(model)

In [ ]:
TEST_REVIEW = ['this is such an amazing movie!']

In [ ]:
BERT_RAW_RESULT = model(tf.constant(TEST_REVIEW))
print(BERT_RAW_RESULT)

### Train and evaluate your BERT sentiment classifier

In [ ]:
HPARAMS.update({
    # TODO: Save your BERT sentiment classifier locally in the form of <key>:<path to save the model>. 
    # Hint: You can use the key as 'model-dir' and save it to './bert-sentiment-classifier-local'.
    "model-dir": './bert-sentiment-classifier-local'
    
})

**Note:** training your model locally will take about 10-15 minutes.

In [ ]:
# history = train_evaluate(HPARAMS)

In [ ]:
# history_dict = history.history
# print(history_dict.keys())

# acc = history_dict['binary_accuracy']
# val_acc = history_dict['val_binary_accuracy']
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(acc) + 1)
# fig = plt.figure(figsize=(10, 6))
# fig.tight_layout()

# plt.subplot(2, 1, 1)
# # "bo" is for "blue dot"
# plt.plot(epochs, loss, 'r', label='Training loss')
# # b is for "solid blue line"
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# # plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(2, 1, 2)
# plt.plot(epochs, acc, 'r', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend(loc='lower right');

## Containerize your model code

Now that you trained and evaluated your model locally in a Vertex Notebook as part of an experimentation workflow, your next step is to train and deploy your model on Google Cloud's Vertex AI platform.

To train your BERT classifier on Google Cloud, you will you will package your Python training scripts and write a Dockerfile that contains instructions on your ML model code, dependencies, and execution instructions. You will build your custom container with Cloud Build, whose instructions are specified in `cloudbuild.yaml` and publish your container to your Artifact Registry. This workflow gives you the opportunity to use the same container to run as part of a portable and scalable [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) workflow. 


You will walk through creating the following project structure for your ML mode code:
```
|--/bert-sentiment-classifier
   |--/trainer
      |--__init__.py
      |--model.py
      |--task.py
   |--Dockerfile
   |--cloudbuild.yaml
   |--requirements.txt
```

## Use Cloud Build to build and submit your model container to Google Cloud Artifact Registry

Next, you will use [Cloud Build](https://cloud.google.com/build) to build and upload your custom TensorFlow model container to [Google Cloud Artifact Registry](https://cloud.google.com/artifact-registry). 

Cloud Build brings reusability and automation to your ML experimentation by enabling you to reliably build, test, and deploy your ML model code as part of a CI/CD workflow. Artifact Registry provides a centralized repository for you to store, manage, and secure your ML container images. This will allow you to securely share your ML work with others and reproduce experiment results.

**Note**: the initial build and submit step will take about 16 minutes but Cloud Build is able to take advantage of caching for faster subsequent builds. 

### 1. Create Artifact Registry for custom container images 

**NOTE:** For any help to create the Artifact Registry, you can refer this [Documentation](https://cloud.google.com/sdk/gcloud/reference/artifacts/repositories/create).

In [ ]:
ARTIFACT_REGISTRY="bert-sentiment-classifier"

In [ ]:
# TODO: create a Docker Artifact Registry using the gcloud CLI. Note the required 'repository-format', 'location' and 'description' flags while creating the Artifact Registry.
# Documentation link: https://cloud.google.com/sdk/gcloud/reference/artifacts/repositories/create
# Create a Docker Artifact Registry using the gcloud CLI with correct parameters


# !gcloud artifacts repositories create bert-sentiment-classifier \
#     --repository-format=docker \
#     --location={REGION} \
#     --description="Docker repository for BERT sentiment classifier"


### 2. Create `cloudbuild.yaml` instructions

In [ ]:
IMAGE_NAME="bert-sentiment-classifier"
IMAGE_TAG="latest"
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REGISTRY}/{IMAGE_NAME}:{IMAGE_TAG}"

In [ ]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
images: 
- '{IMAGE_URI}'"""

with open(f"cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### 3. Build and submit your container image to Artifact Registry using Cloud Build

**Note:** your custom model container will take about 16 minutes initially to build and submit to your Artifact Registry. Artifact Registry is able to take advantage of caching so subsequent builds take about 10 minutes. For any help to submit a build, you can refer this [**documentation**](https://cloud.google.com/sdk/gcloud/reference/builds/submit).

In [ ]:
# TODO: use Cloud Build to build and submit your custom model container to your Artifact Registry.
# Documentation link: https://cloud.google.com/sdk/gcloud/reference/builds/submit
# Hint: make sure the config flag is pointed at `{MODEL_DIR}/cloudbuild.yaml` defined above and you include your model directory as {MODEL_DIR}. Also, add a timeout flag.

# !gcloud builds submit . \
#     --config=./cloudbuild.yaml \
#     --timeout=900s


In [ ]:
from kfp.v2 import compiler
from my_pipeline import pipeline
TEMPLATE_PATH = 'pipeline.json'
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path=TEMPLATE_PATH
)


## Run the pipeline on Vertex Pipelines

The `PipelineJob` is configured below and triggered through the `run()` method.

**Note:** This pipeline run will take around **30-40** minutes to train and deploy your model. Follow along with the execution using the URL from the job output below.

In [ ]:
TIMESTAMP=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
DISPLAY_NAME = "bert-sentiment-{}".format(TIMESTAMP)
GCS_BASE_OUTPUT_DIR= f"{GCS_BUCKET}/bert_sentiment_classifier-{TIMESTAMP}"
SERVING_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest"


In [ ]:
vertex_pipelines_job = aiplatform.PipelineJob(
    display_name="bert-sentiment-classification",
    template_path=TEMPLATE_PATH,
    parameter_values={
        "project": PROJECT_ID,
        "location": REGION,
        "staging_bucket": GCS_BUCKET,
        "display_name": DISPLAY_NAME,        
        "container_uri": IMAGE_URI,
        "model_serving_container_image_uri": SERVING_IMAGE_URI,        
        "base_output_dir": GCS_BASE_OUTPUT_DIR},
    enable_caching=True,
)

In [32]:
vertex_pipelines_job.run()

## Query deployed model on Vertex Endpoint for online predictions

Finally, you will retrieve the `Endpoint` deployed by the pipeline and use it to query your model for online predictions.

Configure the `Endpoint()` function below with the following parameters:

*  `endpoint_name`: A fully-qualified endpoint resource name or endpoint ID. Example: "projects/123/locations/us-central1/endpoints/456" or "456" when project and location are initialized or passed.
*  `project_id`: GCP project.
*  `location`: GCP region.

Call `predict()` to return a prediction for a test review.

In [ ]:
# Retrieve your deployed Endpoint name from your pipeline.
ENDPOINT_NAME = aiplatform.Endpoint.list()[0].name
ENDPOINT_NAME

In [ ]:
#TODO: Generate online predictions using your Vertex Endpoint. 
#Hint: You need to add the following variables: endpoint_name, project, location, with their required values.

endpoint = aiplatform.Endpoint(
    endpoint_name=ENDPOINT_NAME,
    location = REGION,
    project=PROJECT_ID
)

In [ ]:
#TODO: write a movie review to test your model e.g. "The Dark Knight is the best Batman movie!"
test_review = ["This is fantastic"]

In [ ]:
# TODO: use your Endpoint to return prediction for your 'test_review' using 'endpoint.predict()' method.
prediction = endpoint.predict(test_review)

In [ ]:
print(prediction.predictions)

In [ ]:
# Use a sigmoid function to compress your model output between 0 and 1. For binary classification, a threshold of 0.5 is typically applied
# so if the output is >= 0.5 then the predicted sentiment is "Positive" and < 0.5 is a "Negative" prediction.
print(tf.sigmoid(prediction.predictions[0]))

## Next steps

Congratulations! You walked through a full experimentation, containerization, and MLOps workflow on Vertex AI. First, you built, trained, and evaluated a BERT sentiment classifier model in a Vertex Notebook. You then packaged your model code into a Docker container to train on Google Cloud's Vertex AI. Lastly, you defined and ran a Kubeflow Pipeline on Vertex Pipelines that trained and deployed your model container to a Vertex Endpoint that you queried for online predictions.

## License

Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.